In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
df = pd.read_csv("flight_data_2024.csv")
COLUNA_ALVO = 'total_departure_delay'

print(f"--- Iniciando Regressão Linear para Prever o Atraso Total ({COLUNA_ALVO}) ---")

 
    # a) Criação do ID Único
df['flight_id'] = np.arange(df.shape[0]) + 1
    
    # b) Criação e Preparação do NOVO Target (Y)
    
    # Preenchimento de Nulos no weather_delay e late_aircraft_delay (assumindo 0 se nulo)
df['weather_delay'] = pd.to_numeric(df['weather_delay'], errors='coerce').fillna(0)
df['late_aircraft_delay'] = pd.to_numeric(df['late_aircraft_delay'], errors='coerce').fillna(0)
    
    # Cria a coluna de atraso total na saída (soma)
df[COLUNA_ALVO] = df['weather_delay'] + df['late_aircraft_delay']
    
    # c) Preparação de Variáveis Numéricas (taxi_out)
df['taxi_out'] = pd.to_numeric(df['taxi_out'], errors='coerce').fillna(0)
    
    # d) Features de Tempo
df['fl_date'] = pd.to_datetime(df['fl_date'], errors='coerce')
df['day_of_year'] = df['fl_date'].dt.dayofyear
df['week_of_year'] = df['fl_date'].dt.isocalendar().week.astype(int)
    
    # e) Codificação de Variáveis Categóricas
colunas_para_dummies = ['origin', 'origin_state_nm']
df = pd.get_dummies(df, columns=colunas_para_dummies, drop_first=True)
    
    # f) Remoção de colunas que não podem ser Features
colunas_para_remover = [
        'fl_date', 
        'origin_city_name', 
        'flight_id',
        'cancelled', 
        'weather_delay',         # Evitar vazamento de dados (já estão no Target)
        'late_aircraft_delay',   # Evitar vazamento de dados (já estão no Target)
    ]
df.drop(columns=colunas_para_remover, inplace=True, errors='ignore')

    
    # 2. Definição de X e Y
Y = df[COLUNA_ALVO] 
X = df.drop(columns=[COLUNA_ALVO]) 
    
    # Tratamento final: Remoção de NaN/Infinitos e seleção de apenas números
X = X.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
X = X.select_dtypes(include=np.number) 

print(f"Dataset pronto. X: {X.shape[0]} linhas, {X.shape[1]} features.")
    
    # 3. Divisão Treino e Teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
    
print("\n--- Treinando Modelo de Regressão Linear ---")
    
    # 4. Treinamento do Modelo
model_reg = LinearRegression()
model_reg.fit(X_train, Y_train)
    
    # 5. Previsão e Avaliação
Y_pred_reg = model_reg.predict(X_test)
    
print("\n=======================================================")
print(f"✅ RESULTADOS DA REGRESSÃO LINEAR (PREVISÃO DE {COLUNA_ALVO.upper()})")
print("=======================================================")
    
    # Métricas de Desempenho
mse = mean_squared_error(Y_test, Y_pred_reg)
rmse = np.sqrt(mse) 
r2 = r2_score(Y_test, Y_pred_reg)

print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse:.2f} minutos")
print(f"Coeficiente de Determinação (R²): {r2:.4f}")

In [ ]:
df = pd.read_csv("flight_data_2024.csv")

print(f"--- Regressão Linear com NOVAS FEATURES ---")


    # Preparação de Variáveis e Target (Y)
df['flight_id'] = np.arange(df.shape[0]) + 1
df['weather_delay'] = pd.to_numeric(df['weather_delay'], errors='coerce').fillna(0)
df['late_aircraft_delay'] = pd.to_numeric(df['late_aircraft_delay'], errors='coerce').fillna(0)
df['taxi_out'] = pd.to_numeric(df['taxi_out'], errors='coerce').fillna(0)
    
    # TARGET Y (Atraso Total na Saída)
df[COLUNA_ALVO] = df['weather_delay'] + df['late_aircraft_delay']
    
    # -----------------------------------------------------------
    # ⭐ NOVA FEATURE 1: Período do Dia (Usando dep_time)
    # -----------------------------------------------------------
    # dep_time é um número inteiro, e.g., 1000 = 10:00, 1830 = 18:30.
    # Vamos converter para uma coluna de horas e categorizar
df['dep_hour'] = (df['dep_time'] // 100).fillna(-1)
    
bins = [0, 6, 12, 17, 21, 24] # Madrugada, Manhã, Tarde, Noite, Madrugada
labels = ['Madrugada', 'Manha', 'Tarde', 'Noite', 'Madrugada']
df['time_of_day'] = pd.cut(df['dep_hour'], bins=bins, labels=labels, right=False, ordered=False)
df['time_of_day'] = df['time_of_day'].astype(str).replace('nan', 'Desconhecido')
    
    # -----------------------------------------------------------
    # ⭐ NOVA FEATURE 3: Flag de Atraso de Aeronave
    # -----------------------------------------------------------
    # Indicador de se o atraso já existia por causa do avião anterior
df['flag_late_aircraft'] = (df['late_aircraft_delay'] > 0).astype(int)
    
    # --- Feature Engineering Antigo ---
df['fl_date'] = pd.to_datetime(df['fl_date'], errors='coerce')
df['day_of_year'] = df['fl_date'].dt.dayofyear
df['week_of_year'] = df['fl_date'].dt.isocalendar().week.astype(int)
    
    # Codificação de Variáveis Categóricas (INCLUINDO DESTINO e time_of_day)
colunas_para_dummies = ['origin', 'origin_state_nm', 'time_of_day']
df = pd.get_dummies(df, columns=colunas_para_dummies, drop_first=True)
    
    # Remoção de colunas que não podem ser Features
colunas_para_remover = [
        'fl_date', 'origin_city_name', 'flight_id', 'cancelled', 
        COLUNA_ALVO, # Removido para ser definido como Y
        'weather_delay', 'late_aircraft_delay', # Vazamento de dados
        'dep_hour', 'dep_time' # Já foram usados para criar 'time_of_day'
    ]
    
    # Definir X e Y antes de remover o alvo Y
Y = df[COLUNA_ALVO] 
X = df.drop(columns=[c for c in colunas_para_remover if c in df.columns], errors='ignore') 
    
    # Tratamento final: Garantir que X só tem números
X = X.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
X = X.select_dtypes(include=np.number) 
    
print(f"Dataset pronto com novas features. X: {X.shape[0]} linhas, {X.shape[1]} features.")
    
    # 3. Divisão Treino e Teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=35)
    
print("\n--- Treinando Modelo de Regressão Linear APRIMORADO ---")
    
    # 4. Treinamento do Modelo
model_reg_new = LinearRegression()
model_reg_new.fit(X_train, Y_train)
    
    # 5. Previsão e Avaliação
Y_pred_reg_new = model_reg_new.predict(X_test)
    
print("\n=======================================================")
print(f"✅ RESULTADOS DA REGRESSÃO LINEAR COM NOVAS FEATURES")
print("=======================================================")
    
    # Métricas de Desempenho
mse_new = mean_squared_error(Y_test, Y_pred_reg_new)
rmse_new = np.sqrt(mse_new) 
r2_new = r2_score(Y_test, Y_pred_reg_new)

print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse_new:.2f} minutos")
print(f"Coeficiente de Determinação (R²): {r2_new:.4f}")

print(f"\nResultado Anterior: R² = {r2:.4f}")


In [ ]:
print(f"--- Iniciando RANDOM FOREST REGRESSOR para Prever o Atraso Total ---")

df = pd.read_csv("flight_data_2024.csv")
    
    # Preenchimento de Nulos e Criação do Target (Y)
df['flight_id'] = np.arange(df.shape[0]) + 1
df['weather_delay'] = pd.to_numeric(df['weather_delay'], errors='coerce').fillna(0)
df['late_aircraft_delay'] = pd.to_numeric(df['late_aircraft_delay'], errors='coerce').fillna(0)
df['taxi_out'] = pd.to_numeric(df['taxi_out'], errors='coerce').fillna(0)
    
    # TARGET Y (Atraso Total na Saída)
df[COLUNA_ALVO] = df['weather_delay'] + df['late_aircraft_delay']
    
    # Criação das Novas Features: time_of_day e dest
df['dep_hour'] = (df['dep_time'] // 100).fillna(-1)
bins = [0, 6, 12, 17, 21, 24]
labels = ['Madrugada', 'Manha', 'Tarde', 'Noite', 'Madrugada']
df['time_of_day'] = pd.cut(df['dep_hour'], bins=bins, labels=labels, right=False, ordered=False)
df['time_of_day'] = df['time_of_day'].astype(str).replace('nan', 'Desconhecido')
df['flag_late_aircraft'] = (df['late_aircraft_delay'] > 0).astype(int)
    
    # Features de Tempo e Codificação
df['fl_date'] = pd.to_datetime(df['fl_date'], errors='coerce')
df['day_of_year'] = df['fl_date'].dt.dayofyear
df['week_of_year'] = df['fl_date'].dt.isocalendar().week.astype(int)
    
colunas_para_dummies = ['origin', 'origin_state_nm', 'time_of_day'] 
df = pd.get_dummies(df, columns=colunas_para_dummies, drop_first=True)
    
    # Definição de X e Y
colunas_para_remover = [
        'fl_date', 'origin_city_name', 'flight_id', 'cancelled', 
        COLUNA_ALVO, 
        'weather_delay', 'late_aircraft_delay', 
        'dep_hour', 'dep_time'
    ]
    
Y = df[COLUNA_ALVO] 
X = df.drop(columns=[c for c in colunas_para_remover if c in df.columns], errors='ignore') 
    
    # Tratamento final: Garantir que X só tem números
X = X.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
X = X.select_dtypes(include=np.number) 
    
print(f"Dataset pronto. X: {X.shape[0]} linhas, {X.shape[1]} features.")
    
    # 3. Divisão Treino e Teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
    
print("\n--- Treinando Random Forest (Pode demorar um pouco mais...) ---")
    
    # 4. Treinamento do Modelo - Usando menos árvores (n_estimators) para ser mais rápido
model_rf = RandomForestRegressor(
        n_estimators=100,      # Número de árvores
        max_depth=10,          # Profundidade máxima de cada árvore (para evitar overfitting)
        random_state=42, 
        n_jobs=-1              # Usa todos os núcleos da CPU para acelerar
    )
model_rf.fit(X_train, Y_train)
    
    # 5. Previsão e Avaliação
Y_pred_rf = model_rf.predict(X_test)
    
print("\n=======================================================")
print("✅ RESULTADOS DO RANDOM FOREST REGRESSOR")
print("=======================================================")
    
    # Métricas de Desempenho
mse_rf = mean_squared_error(Y_test, Y_pred_rf)
rmse_rf = np.sqrt(mse_rf) 
r2_rf = r2_score(Y_test, Y_pred_rf)

print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse_rf:.2f} minutos")
print(f"Coeficiente de Determinação (R²): {r2_rf:.4f}")

print("\n--- Comparação ---")
print(f"Regressão com features basicas: R² {r2:.4f}")
print(f"Regressão Linear com engenharia de features: R² = {r2_new:.4f}")
print(f"Random Forest Regressor: R² = {r2_rf:.4f}")



### Inferências após a aplicação do modelo de previsão de atrasos de vôos ###

A Regressão Linear e o Random Forest Regressor foram usados para prever o tempo exato do Atraso Total de Partida (total_departure_delay).

- Inferência Principal: O limite de R² em 0.2265 nos diz que o erro médio (RMSE) de cerca de 32 minutos não pode ser reduzido com as features que temos.

- Conclusão de Negócio: O modelo atual não se mostrou eficaz para prever o tempo de atraso, possuindo limitações para que sua implementação seja feita. Para que a previsão de atraso seja útil (ex: R² > 0.60 e RMSE < 15 min), Serão necessarios novos dados de dinâmica operacional na base, como:Tráfego/Volume de Voos. Dados de Chegada do Avião Anterior (o famoso atraso em cascata). Condições Meteorológicas 